In [1]:
import pandas as pd
import gtfs_kit as gk


In [2]:
feed = gk.feed.read_feed('../general_data_analytics.zip',dist_units="km")

In [3]:
routes = gk.routes.get_routes(feed)
stop_times = feed.get_stop_times()
trips = feed.get_trips()
stops = feed.get_stops()
shapes = feed.shapes
calendar_dates = feed.calendar_dates
calendar = feed.calendar

# Collecting a single week data

## Select trips between Mon 02-Oct-2023 and 06-Oct-2023


In [4]:
## Collecting all service date of trips on those days
sw_calendar_dates = calendar_dates.loc[(calendar_dates['date'] == '20231002') | 
                                    (calendar_dates['date'] == '20231003') |
                                    (calendar_dates['date'] == '20231004') |
                                    (calendar_dates['date'] == '20231005') |
                                    (calendar_dates['date'] == '20231006')]


In [8]:
## Creating new calendar and collect service date from previous new calendar_dates with drop duplicates
sw_calendar = calendar.iloc[0:0]
service_id_list = list(dict.fromkeys( sw_calendar_dates['service_id']))

for service_id in service_id_list:
    row  = calendar.loc[calendar['service_id'] == service_id]
    sw_calendar = pd.concat([sw_calendar, row],
                      ignore_index = True)

In [9]:
## Creating new trips dataframe and collect trips using the list of service id
sw_trips = trips.iloc[0:0]

for service_id in service_id_list:
    row  = trips.loc[trips['service_id'] == service_id]
    sw_trips = pd.concat([sw_trips, row],
                      ignore_index = True)

In [12]:
## Creating list of shapes_id,route_id,trips_id from sw_trips
route_id_list = list(dict.fromkeys(sw_trips['route_id']))
shape_id_list = list(dict.fromkeys(sw_trips['shape_id']))
trip_id_list = list(dict.fromkeys(sw_trips['trip_id']))

In [14]:
## Creating new route dataframe
sw_routes = routes.iloc[0:0]

for route_id in route_id_list:
    row = routes.loc[routes['route_id']==route_id]
    sw_routes = pd.concat([sw_routes,row],ignore_index=True)

In [17]:
## Creating new shape dataframe
sw_shapes = shapes.iloc[0:0]

for shape_id in shape_id_list:
    row = shapes.loc[shapes['shape_id']==shape_id]
    sw_shapes = pd.concat([sw_shapes, row],ignore_index = True)

In [16]:
## Creating new stop_times dataframe
sw_stop_times = stop_times.iloc[0:0]

for trip_id in trip_id_list:
    row = stop_times.loc[stop_times['trip_id'] == trip_id]
    sw_stop_times = pd.concat([sw_stop_times,row],ignore_index = True)

In [28]:
stop_id_list = list(dict.fromkeys(sw_stop_times['stop_id']))

In [31]:
## Creating new stop dataframe, should be equal at this point 
## but we do it for safe reasons
sw_stops = stops.iloc[0:0]

for stop_id in stop_id_list:
    row = stops.loc[stops['stop_id']==stop_id]
    sw_stops = pd.concat([sw_stops,row],ignore_index = True)


(28, 6)
(28, 6)


In [32]:
# Check value change
subtracted_dimensions = pd.DataFrame(columns=["Name","Initial","Updated","Difference"])


res = tuple(map(lambda i, j: i - j, routes.shape, sw_routes.shape))
subtracted_dimensions.loc[0] = ["Routes",routes.shape, sw_routes.shape, res]

res = tuple(map(lambda i, j: i - j, stops.shape, sw_stops.shape))
subtracted_dimensions.loc[1] = ["Stops",stops.shape, sw_stops.shape, res]

res = tuple(map(lambda i, j: i - j, trips.shape, sw_trips.shape))
subtracted_dimensions.loc[2] = ["Trips",trips.shape, sw_trips.shape, res]

res = tuple(map(lambda i, j: i - j, stop_times.shape,sw_stop_times.shape))
subtracted_dimensions.loc[3] = ["Stop_Times",stop_times.shape, sw_stop_times.shape, res]

res = tuple(map(lambda i, j: i - j, shapes.shape, sw_shapes.shape))
subtracted_dimensions.loc[4] = ["Shapes",shapes.shape, sw_shapes.shape, res]

res = tuple(map(lambda i, j: i - j, calendar.shape, sw_calendar.shape))
subtracted_dimensions.loc[5] = ["Calendar",calendar.shape, sw_calendar.shape, res]

res = tuple(map(lambda i, j: i - j, calendar_dates.shape, sw_calendar_dates.shape))
subtracted_dimensions.loc[6] = ["Calendar_Dates",calendar_dates.shape, sw_calendar_dates.shape, res]

subtracted_dimensions

,Name,Initial,Updated,Difference
0,Routes,"(3, 7)","(1, 7)","(2, 0)"
1,Stops,"(28, 6)","(28, 6)","(0, 0)"
2,Trips,"(1149, 7)","(51, 7)","(1098, 0)"
3,Stop_Times,"(12393, 6)","(654, 6)","(11739, 0)"
4,Shapes,"(96692, 6)","(41916, 6)","(54776, 0)"
5,Calendar,"(201, 10)","(13, 10)","(188, 0)"
6,Calendar_Dates,"(28260, 2)","(65, 2)","(28195, 0)"
